**Partie 1 : visualisation des pistes cyclables**


In [ ]:
!pip install geopandas

In [ ]:
!pip install folium

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import folium
import os

In [ ]:
amenagements = gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/33d97256-1829-41d3-b7b5-56e490846b85")
#https://www.data.gouv.fr/fr/datasets/amenagements-velo-en-ile-de-france-2/#_

In [ ]:
#pour simplifier on elève les pistes qui ne sont pas à double sens 
am = amenagements.dropna(subset = ['ad','ag']) 
#am.count()
#on perd 1000 observations à peu près 
#am['ad'].unique()

In [ ]:
#on simplifie les types de pistes
conditionlist = [
                 (am['ad'].isin(['chemin service site propre uni','chemin dedie uni','autre chemin velo uni'])), 
                 (am['ad'].isin(['cheminement trottoir uni','piste trottoir uni'])),
                 (am['ad'].isin(['piste uni', 'piste bi'])),
                 (am['ad'].isin(['bande uni', 'bande bi', 'DSC bande'])),
                 (am['ad'].isin(['voie verte uni'])),
                 (am['ad'].isin(['voie bus uni']))]
choicelist = ['voie propre', 'trottoir', 'piste', 'bande', 'voie verte', 'voie bus']
am['type'] = np.select(conditionlist, choicelist, default = 'Not Specified')

/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [ ]:
conditionlist = [
                 (am['type'].isin(['voie propre'])), 
                 (am['type'].isin(['piste'])),
                 (am['type'].isin(['voie verte'])),
                 (am['type'].isin(['bande'])),
                 (am['type'].isin(['trottoir'])),
                 (am['type'].isin(['voie bus']))]
choicelist = [1, 2, 3, 4, 5, 6]
am['idtype'] = np.select(conditionlist, choicelist, default = 'Not Specified')

/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [ ]:
am['idtype'] = am['idtype'].str.replace(',','.').astype(float)
#am.dtypes
am = am.to_json()

/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [ ]:
centre = [48.8534 , 2.3488] #le centre de paris 
idf = folium.Map(location = centre, zoom_start = 11) #zoom = 11 permet de voir toute l'idf, à modifier en fct de l'utilisation de l'appli 
#idf.add_child(folium.GeoJson(am))

In [ ]:
idf


> Là j'arrive pas à afficher joliment les pistes sur folium...


In [ ]:
#idf.add_child(folium.GeoJson(am, style_function = lambda x : {'fillcolor' : '#00ff00' if x['properties']['type']=='voie verte' else '#00ff00'}))
#idf
#folium.GeoJson(data, style_function=None, name=None, overlay=True, control=True, show=True, smooth_factor=None, highlight_function=None, tooltip=None)

In [ ]:
#idf.add_child(folium.GeoJson(am, style_function = lambda x : {'color' : '#ff0000' if x['properties']['type']=='piste' 
#                                                              else '#00ff00'}))

In [ ]:
#idf.add_child(folium.GeoJson(am, style_function = lambda x : {'color' : '000000'  if x['properties']['type']=='voie propre' 
 #                                                             else '#ff0000' if x['properties']['type']=='trottoir' 
  #                                                            else '#00ffff' if x['properties']['type']=='piste' 
  #                                                            else '#00ff00' if x['properties']['type']=='voie verte' 
   #                                                           else '#ffff00' if x['properties']['type']=='bande' 
    #                                                          else '#ff00ff'}))


**Partie 2 : calcul d'itinéraire**

In [ ]:
!pip install pyroutelib3

In [ ]:
import pyroutelib3
from pyroutelib3 import Router
import requests, json
import urllib.parse

In [ ]:
import tkinter as tk
from tkinter import simpledialog
from tkinter import messagebox

In [ ]:
#ne marche pas sur googlecolab mais marche sur jupyter
application_window = tk.Tk()
adresse_depart = simpledialog.askstring("Adresse départ", "D'où partez-vous ? (numéro, rue, code postal ville)",
                                parent=application_window)
adresse_arrivee = simpledialog.askstring("Adresse arrivée", "Où voulez-vous aller ? (numéro, rue, code postal ville)",
                                parent=application_window)

In [ ]:
#est-ce qu'on ne pourrait pas plutôt calculer les deux trajets automatiquement et proposer les deux à la fin ? 
a = messagebox.askquestion("Type de trajet", "Préférez-vous un trajet rapide plutôt qu'un trajet agréable?")
if a == 'yes':
  #on recherche le trajet le plus rapide
else : 
  #on recherche le trajet le plus agréable

In [ ]:
#adresse_depart = "57, avenue du Plessis, 92350 Le Plessis Robinson"
#adresse_arrivee = "3, avenue de Verdun, 92120 Montrouge"
dep_json = requests.get("https://api-adresse.data.gouv.fr/search/?q=" + urllib.parse.quote(adresse_depart) + "&format=json").json()
arr_json = requests.get("https://api-adresse.data.gouv.fr/search/?q=" + urllib.parse.quote(adresse_arrivee) + "&format=json").json()
coord_dep = list(dep_json['features'][0].get('geometry').get('coordinates'))
coord_arr = list(arr_json['features'][0].get('geometry').get('coordinates'))
coord_dep
coord_arr

[2.316016, 48.812058]

In [ ]:
coor_depart = [coord_dep[1], coord_dep[0]] #[48.782792,2.278729] # chez mes parents
coor_arrivee = [coord_arr[1], coord_arr[0]] #[48.833428,2.346074] #chez moi

folium.Marker(coor_depart,popup="Départ").add_to(idf)
folium.Marker(coor_arrivee,popup="Arrivée").add_to(idf)

router = pyroutelib3.Router("cycle")
depart = router.findNode(coor_depart[0], coor_depart[1])
print(depart)
arrivee = router.findNode(coor_arrivee[0], coor_arrivee[1])
print(arrivee)

routeLatLons=[coor_depart,coor_arrivee]
status, route = router.doRoute(depart, arrivee)
if status == 'success':
  print("votre trajet existe")
  routeLatLons = list(map(router.nodeLatLon, route))
else:
  print("votre trajet n'existe pas")


print(routeLatLons)
for coord in routeLatLons:
  coord=list(coord)
  folium.CircleMarker(coord,radius = 3,fill=True, color='green').add_to(idf)

folium.PolyLine(routeLatLons, color="red", weight=2.5, opacity=1).add_to(idf)

4071188702
6407463344
votre trajet existe
[(48.7828824, 2.278604), (48.7835111, 2.2789991), (48.7835701, 2.281373), (48.7835841, 2.2816924), (48.7835842, 2.2817545), (48.7835842, 2.2818166), (48.7835424, 2.2818332), (48.7835058, 2.2818677), (48.7834775, 2.2819171), (48.7834601, 2.281977), (48.7834551, 2.282042), (48.7834633, 2.2821079), (48.7834842, 2.282167), (48.7835159, 2.2822137), (48.7835554, 2.2822437), (48.7836006, 2.2822541), (48.7836457, 2.2822427), (48.7836862, 2.2822104), (48.783718, 2.2821606), (48.7837384, 2.2820962), (48.783744, 2.2820253), (48.7837344, 2.2819554), (48.7837611, 2.2819501), (48.7837879, 2.2819448), (48.7841188, 2.2817591), (48.7846419, 2.2814736), (48.7848223, 2.2814719), (48.7849286, 2.2815104), (48.7851936, 2.2817354), (48.7855305, 2.2820955), (48.7855751, 2.2821426), (48.7856293, 2.2822), (48.7858208, 2.2824164), (48.7865538, 2.2832092), (48.7866385, 2.2833209), (48.7868437, 2.28345), (48.7870563, 2.2836198), (48.7875922, 2.2839472), (48.7875791, 2.2840

In [ ]:
idf


> on calcule la distance du trajet 


In [ ]:
distance = 0
for x in range(len(routeLatLons)-1):
  a = (max(routeLatLons[x][0], routeLatLons[x+1][0]) - min(routeLatLons[x][0], routeLatLons[x+1][0]))**2 + (max(routeLatLons[x][1], routeLatLons[x+1][1]) - min(routeLatLons[x][1], routeLatLons[x+1][1]))**2
  a = a**(1/2)*100
  distance = distance + a
print("Votre trajet est de", distance, "km")

Votre trajet est de 6.470746424052413 km
